In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
#load the data
violations_df = pd.read_csv('health_inspect_cleaned_loc_added.csv', index_col=0) # file is in GitHub 

In [3]:
vdf=violations_df[~pd.isnull(violations_df['latitude'])] 
len(vdf)

22307

### Finding number of complains in 311 dataset

In [2]:
#import data
complains_311_filtered = pd.read_csv('311_Service_Requests_from_last_year_filted.csv', index_col=0) # file is in GitHub 
#complains_311_filtered

/Users/motorrecoveryresearchlab/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#convert to datetime object 
date_format = "%Y-%m-%d" 
date_format2 = "%m/%d/%Y"
vdf['latest_inspection']= pd.to_datetime(vdf['latest_inspection']) 
complains_311_filtered['Created Date']= pd.to_datetime(complains_311_filtered['Created Date']) 


In [4]:
complains_311_filtered=complains_311_filtered[~pd.isnull(complains_311_filtered['Latitude'])] 
len(complains_311_filtered)

458503

In [5]:
#break down the complaints 
rodent=complains_311_filtered[complains_311_filtered['Complaint Type']=='Rodent'] 
dirty_conditions=complains_311_filtered[complains_311_filtered['Complaint Type']=='Dirty Conditions']
electric=complains_311_filtered[complains_311_filtered['Complaint Type']=='ELECTRIC']
food_establishment=complains_311_filtered[complains_311_filtered['Complaint Type']=='Food Establishment']
general=complains_311_filtered[complains_311_filtered['Complaint Type']=='GENERAL']
missed_collection=complains_311_filtered[complains_311_filtered['Complaint Type']=='Missed Collection (All Materials)']

noise=complains_311_filtered[complains_311_filtered['Complaint Type']=='Noise']
police_matter=complains_311_filtered[complains_311_filtered['Complaint Type']=='Non-Emergency Police Matter']
plumbing=complains_311_filtered[(complains_311_filtered['Complaint Type']=='PLUMBING')| (complains_311_filtered['Complaint Type']=='Plumbing')]
safety=complains_311_filtered[complains_311_filtered['Complaint Type']=='SAFETY']
sanitary=complains_311_filtered[(complains_311_filtered['Complaint Type']=='Sanitation Condition') | (complains_311_filtered['Complaint Type']=='UNSANITARY CONDITION')|(complains_311_filtered['Complaint Type']=='WATER LEAK')| (complains_311_filtered['Complaint Type']=='Sewer')]
food_poisoning=complains_311_filtered[complains_311_filtered['Complaint Type']=='Food Poisoning']



In [8]:
print(len(rodent)) 
print(len(dirty_conditions)) 
print(len(electric))
print(len(food_establishment))
print(len(general))
print(len(missed_collection))
print(len(noise))
print(len(police_matter))
print(len(plumbing)) 
print(len(safety)) 
print(len(sanitary)) 
print(len(food_poisoning))

31598
33837
27509
9811
25772
30306
55468
8868
55097
8000
174625
3067


In [20]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


In [ ]:
food_poisoning 
#only inlcude the restaurant poisoning
food_poisoning=food_poisoning[food_poisoning['Location Type']=='Restaurant/Bar/Deli/Bakery']

In [58]:
#food_poisoning
from tqdm import tqdm 

#initilize arrays
complain_count = []

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets

for index, row in (vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in (food_poisoning.iterrows()):
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1 
                
    complain_count.append(cc)
  
vdf['food_poisoning_complains'] = complain_count


/Users/motorrecoveryresearchlab/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
food_establishment=food_establishment[food_establishment['Location Type']=='Restaurant/Bar/Deli/Bakery']
len(food_establishment)
#food_establishment

8584

In [ ]:
#food_establishment
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in food_establishment.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['food_establishment_complains'] = complain_count 


3it [00:06,  2.08s/it]

In [ ]:
#rodent
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in rodent.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1 
                
    complain_count.append(cc)
  
vdf['rodent_complains'] = complain_count 


In [ ]:
#dirty conditions
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in dirty_conditions.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1 
                
    complain_count.append(cc)
  
vdf['dirty_conditions_complains'] = complain_count 


In [ ]:
#electric
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in electric.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['electric_complains'] = complain_count 


In [ ]:
#general
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in general.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1 
                
    complain_count.append(cc)
  
vdf['general_complains'] = complain_count 


In [ ]:
#missed collection
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in missed_collection.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['missed_collection_complains'] = complain_count 


In [ ]:
#noise
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in noise.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['noise_complains'] = complain_count 


In [ ]:
#police matter
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in police_matter.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['police_matter'] = complain_count 


In [ ]:
#plumbing
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in plumbing.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['plumbing'] = complain_count 


In [ ]:
#safety
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in safety.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1 
                
    complain_count.append(cc)
  
vdf['safety'] = complain_count 


In [ ]:
#sanitary
from tqdm import tqdm 

#date format for date comparison
from datetime import datetime  

#initilize arrays
complain_count = []
lat1 = []
lon1 = []
lat2 = []
lon2 = [] 

#loop through all the entries in the violation data frame 
radius = 0.30 # in kilometer, around 3 streets


for index, row in tqdm(vdf.iterrows()):
 
    #loop through all the entries in the 311 data frame 
    cc=0
    for index2, row2 in sanitary.iterrows():
        a = haversine(row['longitude'],row['latitude'], row2['Longitude'], row2['Latitude']) 
         
        #check if the complain is inside the circle
        if a < radius:  
            
            cc=cc+1  
                
    complain_count.append(cc)
  
vdf['sanitary'] = complain_count 


In [ ]:
vdf.to_csv('vdf_complains_added.csv')